In [1]:
import pandas as pd
import datetime
import requests
import json
import csv
# import matplotlib as plt

#-*- coding: utf-8 -*-

# kim key no.1 : c15fcbaab76cc525e9342c5e4dda712f
# kim key no.2 : c3268b0918814ae0a2bc298f7ba893a3
# lee key no.1 : 77a87d41d6e2cbe3131a190d5c48514e 
# lee key no.2 : e53650978fc97b47c7f932136ce2c08a

In [3]:
key1 = 'e53650978fc97b47c7f932136ce2c08a'

def crawl_movie_boxoffice_data(start_date, end_date, key = key1) :
    result_list = []
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json'
    
    for single_date in pd.date_range(start_date, end_date) :
        for multi, rep in zip(['Y', 'Y', 'N', 'N'], ['K', 'F', 'K', 'F']) :
            payload = {
                'key' : key,
                'targetDt' : single_date.strftime('%Y%m%d'),
                'itemPerPage' : '10',
                'multiMovieYn' : multi,
                'repNationCd' : rep
            }
            
            req = requests.get(url, params = payload)

            for item in req.json()['boxOfficeResult']['dailyBoxOfficeList'] :
                temp_list = []
                header_list = []
                for header, value in item.items() :
                    header_list.append(header)
                    temp_list.append(value)
                    
                temp_list.append(single_date)
                header_list.append('CurrentDate')
                
                temp_list.append(multi)
                header_list.append('multi')
                
                temp_list.append(rep)
                header_list.append('Nation')
                
                result_list.append(temp_list)
                
    return pd.DataFrame(result_list, columns = header_list)

movie_boxoffice_list = crawl_movie_boxoffice_data('20100101', '20101231', key = key1)
movie_boxoffice_list.to_csv('movie_boxoffice_2010.csv', index = False)

In [25]:
all_filenames = []
for i in range(2013, 2019) :
    all_filenames.append('movie_boxoffice_'+str(i)+'.csv')
movie_boxoffice_combined = pd.concat([pd.read_csv(f) for f in all_filenames ])

movie_boxoffice_combined.to_csv('movie_boxoffice.csv', index = False, encoding = 'utf-8')

In [19]:
movie_boxoffice_file_2017 = open('movie_boxoffice_2017.csv', 'r', encoding = 'utf-8')
movie_boxoffice_file_2018 = open('movie_boxoffice_2018.csv', 'r', encoding = 'utf-8')

fa=pd.read_csv(movie_boxoffice_file_2017)
fb=pd.read_csv(movie_boxoffice_file_2018)

print(len(fa))
print(len(fb))
with open('movie_boxoffice.csv', 'r', encoding = 'utf-8') as infile :
    fc=pd.read_csv(infile)
    print(len(fc))

14446
14560
29006


In [26]:
movie_boxoffice_list = pd.read_csv('movie_boxoffice.csv')

len(movie_boxoffice_list)

86747

In [27]:
# movie_rating_list = pd.DataFrame({'movieCd' : movie_boxoffice_f.movieCd.drop_duplicates(), 'rating' : 0})
movie_rating_list = pd.DataFrame({'movieCd' : movie_boxoffice_list.movieCd.drop_duplicates(), 'rating' : 0})

len(movie_rating_list)

6195

In [28]:
movie_rating_list.to_csv('movie_rating_list.csv', index = False, encoding = 'utf-8')

In [45]:
## sorting movie_rating_list
# and divide 3 parts to use seperate keys

movie_rating_list=movie_rating_list.sort_values(by='movieCd', ascending=True).reset_index()
mrl1 = movie_rating_list.iloc[0:3000]
mrl2 = movie_rating_list.iloc[3000:6000]
mrl3 = movie_rating_list.iloc[6000:]



3000
3000
195


In [50]:
key2 = '77a87d41d6e2cbe3131a190d5c48514e'

def crawl_movie_detail_data(lists, key = key2) :
    movie_detail_json = dict()
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'
    
    for movieCd in lists['movieCd'] :
        payload = {
            'key' : key,
            'movieCd' : movieCd
        }
        req = requests.get(url, params = payload)
        
        try :
            movie_detail_json[movieCd] = req.json()['movieInfoResult']['movieInfo']
        except :
            print("Error !")
            print(req.json())
        
    return movie_detail_json

movie_detail = crawl_movie_detail_data(mrl3)

with open('movie_detail_3.json', 'w', encoding = 'utf-8') as output :
    json.dump(movie_detail, output)

In [11]:
with open('movie_detail_1.json', 'r') as f :
    movie_detail_1 = json.load(f)
with open('movie_detail_2.json', 'r') as f :
    movie_detail_2 = json.load(f)
with open('movie_detail_3.json', 'r') as f :
    movie_detail_3 = json.load(f)

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [29]:
## concatenate movie detail(json type) files
# 2013 ~ 2018

movie_detail = dict()
for j in movie_detail_1 :
    movie_detail[j] = movie_detail_1[j]

print(len(movie_detail))
for j in movie_detail_2 :
    movie_detail[j] = movie_detail_2[j]

print(len(movie_detail))
for j in movie_detail_3 :
    movie_detail[j] = movie_detail_3[j]
    
print(len(movie_detail))

with open('movie_detail', 'w', encoding = 'utf-8') as output :
    json.dump(movie_detail, output)

3000
6000
6195


In [ ]:
# movie_detail = json.load('movie_detail.json')

for i in range(len(movie_rating_list)) :
    url = 'https://openapi.naver.com/v1/search/movie.json'
    payload = {
        'query' : movie_detail[movie_rating_list.iloc[i, 0]]['movieNm']
        , 'display' : 10
    }
    
    headers = {
        'X-Naver-Client-Id ' : 'V8LONuGkI0U72r0RY3yo',
        'X-Naver-Client-Secret' : 'VKW2Diwq5d'
    }
    
    if (movie_rating_list.iloc[i, 1] == 0) | (movie_rating_list.iloc[i, 1] == 'NA') :
        try :
            req = requests.get(url, headers = headers, params = payload)
            req.json()['total']
        except :
            movie_rating_list.iloc[i, 1] = 'NA'
        else :
            if(req.json()['total'] == 1) :
                movie_rating_list.iloc[i, 1] = req.json()['items'][0]['userRating']
            else :
                d = movie_detail[movie_rating_list.iloc[i, 0]]['directors']
                flag = False
                for jj in d :
                    for j in req.json()['items'] :
                        if(j['director'][:-1] == jj['peopleNm']) :
                            movie_rating_list.iloc[i,1]=j['userRating']
                            flag = True
                        if(flag == True) :
                            break
                    if(flag == True) :
                        break

NameError: name 'mrl1' is not defined